In [2]:
pip install scikit-learn

  Using cached scikit_learn-1.6.1-cp313-cp313-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached scipy-1.15.2-cp313-cp313-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp313-cp313-macosx_12_0_arm64.whl (11.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp313-cp313-macosx_14_0_arm64.whl (22.4 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.0/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd

# Load your cleaned file
file_path = "/Users/jinenmodi/ImpData/Crypto Sentiment Prediction/crypto_sentiment_project/data/crypto_news_with_sentiment.xlsx"
df = pd.read_excel(file_path)

# Check column names
print("🧾 Available Columns:")
print(df.columns)

🧾 Available Columns:
Index(['source', 'title', 'url', 'published_date', 'scraped_at', 'sentiment'], dtype='object')


In [5]:
pip install vaderSentiment


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.0/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# 📂 Load the cleaned dataset
file_path = "/Users/jinenmodi/ImpData/Crypto Sentiment Prediction/crypto_sentiment_project/data/crypto_news_with_sentiment.xlsx"
df = pd.read_excel(file_path)

# 🔍 Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# 🧠 Define function to classify sentiment
def classify_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "Neutral"  # Handle empty or non-string entries
    score = analyzer.polarity_scores(text)["compound"]
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

# 🏷️ Apply sentiment classification on 'title'
df["sentiment"] = df["title"].apply(classify_sentiment)

# 📊 Show sentiment counts
print(df["sentiment"].value_counts())

# 💾 Save the new file with sentiment column
output_path = "/Users/jinenmodi/ImpData/Crypto Sentiment Prediction/crypto_sentiment_project/data/crypto_news_with_sentiment.xlsx"
df.to_excel(output_path, index=False)
print(f"✅ Sentiment-labeled data saved to: {output_path}")

sentiment
Neutral     1197
Positive     635
Negative     544
Name: count, dtype: int64
✅ Sentiment-labeled data saved to: /Users/jinenmodi/ImpData/Crypto Sentiment Prediction/crypto_sentiment_project/data/crypto_news_with_sentiment.xlsx


In [14]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load your cleaned Excel file
file_path = "/Users/jinenmodi/ImpData/Crypto Sentiment Prediction/crypto_sentiment_project/data/crypto_news_with_sentiment.xlsx"

try:
    df = pd.read_excel(file_path)
    print("File loaded successfully.")
except FileNotFoundError:
    raise FileNotFoundError(f"❌ File not found at: {file_path}")

# Check if 'sentiment' column exists
if 'sentiment' not in df.columns:
    raise KeyError(" 'sentiment' column not found. Please check your dataset.")

# Display unique sentiment values
print(f" Unique sentiment classes: {df['sentiment'].unique()}")

# Initialize and fit LabelEncoder
label_encoder = LabelEncoder()
df['sentiment_encoded'] = label_encoder.fit_transform(df['sentiment'].astype(str))

# Create a mapping dictionary (for interpretation)
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

#  Save encoded DataFrame (optional)
encoded_path = file_path.replace(".xlsx", "_encoded.xlsx")
df.to_excel(encoded_path, index=False)
print(f"Encoded dataset saved to: {encoded_path}")

# Preview sample
print("\n Encoded Sample:")
print(df[['sentiment', 'sentiment_encoded']].head())

File loaded successfully.
 Unique sentiment classes: ['Neutral' 'Negative' 'Positive']
Label Mapping: {'Negative': np.int64(0), 'Neutral': np.int64(1), 'Positive': np.int64(2)}
Encoded dataset saved to: /Users/jinenmodi/ImpData/Crypto Sentiment Prediction/crypto_sentiment_project/data/crypto_news_with_sentiment_encoded.xlsx

 Encoded Sample:
  sentiment  sentiment_encoded
0   Neutral                  1
1   Neutral                  1
2  Negative                  0
3   Neutral                  1
4  Negative                  0


In [13]:
import pandas as pd
import numpy as np
import string
import re
import joblib

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Step 1: Load the cleaned dataset
file_path = "/Users/jinenmodi/ImpData/Crypto Sentiment Prediction/crypto_sentiment_project/data/crypto_news_with_sentiment_encoded.xlsx"
df = pd.read_excel(file_path)

# Step 2: Text Cleaning Function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

# Step 3: Apply cleaning
df['clean_text'] = df['title'].astype(str).apply(clean_text)

# Step 4: TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['clean_text'])

# Step 5: Labels
y = df['sentiment_encoded']

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Model Training
model = MultinomialNB()
model.fit(X_train, y_train)

# Step 8: Evaluation
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Step 9: Save model and vectorizer
joblib.dump(model, "sentiment_model_nb.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
print(" Model and vectorizer saved successfully.")

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.22      0.34        93
           1       0.56      0.98      0.71       242
           2       0.85      0.16      0.27       141

    accuracy                           0.59       476
   macro avg       0.75      0.45      0.44       476
weighted avg       0.70      0.59      0.51       476

Accuracy: 0.5903361344537815
Confusion Matrix:
 [[ 20  70   3]
 [  3 238   1]
 [  1 117  23]]
 Model and vectorizer saved successfully.
